# RoPE
## 前提知識
まずattention機構は次のように計算される。
$$ 
\bm{q}_m = f_q(\bm{x}_m, m) \\
\bm{k}_n = f_k(\bm{x}_n, n) \\
\bm{v}_n = f_v(\bm{x}_n, n) 
$$
入力ベクトルから上の値query, key, valueを定義。
$$
a_{m,n} = \textrm{softmax}({\frac{\bm{q}_m^T \bm{k}_n }{ \sqrt{d}}}) \\
\bm{o}_m = \sum^N_{n=1}a_{m,n}v_n
$$
RNNでは反復計算により入力ベクトルの順番から単語の位置の学習が行われたが、attentionを用いたアーキテクチャでは処理を並列で行うため単語の位置の学習が行われない。そのため入力ベクトルへの位置情報の付加(位置埋め込み:position embedding)が必要である。<br>

## APE: Absolute position embedding（絶対位置埋め込み）
各ベクトルにその語順の絶対的な位置を付加することをAbsolute Positional Embeddingという。
### sinusoidal position embedding
上の問題に対処するためにAttention is All You Needでは次のように正弦波によるpositional embeddingを行った。入力ベクトル$\bm{x}_i$($i\in\{0,1,2,\dots,(d/2)-1\}$)に対し、
$$
f_{t:t\in\{ q,t,k\}}(\bm{x}_i, i) = W_{t:t\in\{ q,t,k\}}(\bm{x}_i+PE_i)
$$
ただし
$$
\begin{align}
PE_{i, 2t} = \sin({i/10000^{2t/d}})\\ PE_{i, 2t+1} = \cos({i/10000^{2t/d}})
\end{align}
$$
$PE_i$は入力ベクトルと同じ次元数であり、位置iに対して一意のベクトルである。これを実際に記述すると、
$$ PE = [\sin(i), \cos(i), \sin(\frac{i}{10000^{2/d}}), \cos(\frac{i}{10000^{2/d}}), \dots, \sin(\frac{i}{10000^{(d-2)/d}}), \cos(\frac{i}{10000^{(d-2)/d}})] $$
![APE](img/APE.png)<br>
三角関数を使う理由は収束と発散をせず、cosも用いる理由は違いを示すために周波数を小さくしたとき、sinの勾配が小さい時でもcosの勾配は大きくなるため。<br>
attentionでは$\bm{q}^T\bm{k}$が単語間のベクトルの類似度を測る役割をしている（詳しくはattention matrix）。ここで行列計算として今回の計算をみると、
$$
QK^T = X \times W_Q \times W_K^T \times X^T = X W X^T
$$
ここでWをひとまず考えずt番目の行とt'番目の列（二つのトークンの内積）を計算すると、
$$
X_tX_{t'}^T=(X_t+PE(t))(X_{t'}+PE(t'))^T=X_tX_{t'}^T+PE(t)X_{t'}+X_tPE(t')+PE(t)PE(t')
$$
ここで$X_tX_{t'}^T$は単純な単語の類似度を示しており、また、$PE(t)PE(t')$は
$$
PE(t)PE(t') = \sin(t)\sin(t')+\cos(t)\cos(t')+ \dots + \sin(\frac{t}{10000^{(d-2)/d}})\sin(\frac{t'}{10000^{(d-2)/d}})+\cos(\frac{t}{10000^{(d-2)/d}})\cos(\frac{t'}{10000^{(d-2)/d}}) \\
= \sum\cos(\frac{1}{10000^{2i/d}}(t-t'))
$$
となりtとt'の位相の違いが表現できる。そのためsinusoidal position embeddingは2項、3項はわからないものの部分的に相対的な位置を表現できる可能性がある。

## Relative position embedding: 相対位置埋め込み
各ベクトルに相対的に考えられるように付加する方法が相対位置埋め込みである。<br>
先ほどの式から2項と3項がないものを考えればより相対的な位置埋め込みが可能であると考えられる。つまり、$X$に先に可算をせず、$X_t X_{t'}$の積の後から位置情報を加算することで第2,3項を無くして考えられる（第4項の情報を形を変えて後から追加する）。
$$
QK^T = XW_QW_K^TX^T +b_{i,j}
$$
まだ詳しく調べてないので後ほど。

## RoPE: Rotary position embedding（回転位置埋め込み）
RoPEは相対位置埋め込みの一種だが、今までの延長線上のものではなく、queryとkeyに回転行列をかけて相対的な埋め込みをする。<br>
まず、
$$
QK^T = XW_QR^{m-n}W_k^TX^T
$$
となるような時期が好ましい。そのため、次のような性質を満たす関数gとなるようなものを考えたい。
$$
\langle f_q(\bm{x}_m, m), f_k(\bm{x}_n, n) \rangle = g(\bm{x}_m, \bm{x}_n, m-n)
$$

### 二次の場合から考える
次元数$d=2$、$\thetaを0でない定数とする$
$$
f_q(\bm{x}_m, m) = (\bm{W}_q\bm{x}_m)e^{im\theta}\\
f_k(\bm{x}_n, n) = (\bm{W}_k\bm{x}_n)e^{in\theta}\\
g(\bm{x}_m, \bm{x}_n, m-n) = \textrm{Re}[(\bm{W}_q\bm{x}_m)\bm{W}_k\bm{x}_ne^{i(m-n)\theta}]
$$
そして$f_{\{q,k\}}$を二次の場合に考えると
$$
f_{\{q,k\}}(x_m, m) =
\begin{pmatrix}
\cos(m\theta) & -\sin(m\theta) \\
\sin(m\theta) & \cos(m\theta)
\end{pmatrix}
\begin{pmatrix}
W_{\{q,k\}}^{(11)} & W_{\{q,k\}}^{(12)} \\
W_{\{q,k\}}^{(21)} & W_{\{q,k\}}^{(22)}
\end{pmatrix}
\begin{pmatrix}
x_m^{(1)} \\
x_m^{(2)}
\end{pmatrix}
$$
とする。

### 二次から一般形への拡張

## 参考url
- https://arxiv.org/abs/2104.09864
- https://www.youtube.com/watch?v=SMBkImDWOyQ (概要、おすすめ)
- https://www.nomuyu.com/positional-encoding/　(APE、おすすめ)
